# import libs

In [ ]:
import os
import numpy as np
import pandas as pd
import torch
from train import TwoInputFCNN as FCNN


# import data

In [ ]:
# paths pre-setting
data_root = '/home/ysy/ERIE/single_psm_manipulation_dataset/output'
model_root = '/home/ysy/ERIE/videos'
model_name = 'DLC_resnet50_TrackJun13shuffle1_50000'


In [ ]:
# pre setting training sets
training_sets = ['C_M1_T1_1', 'C_M1_T1_3', 'C_M1_T1_5', 'C_M1_T1_7', 
                 'R2_M1_T1_1', 'R2_M1_T1_3', 'R2_M1_T1_5', 'R2_M1_T1_7', 
                 'L2_M1_T1_1', 'L2_M1_T1_3', 'L2_M1_T1_5', 'L2_M1_T1_7', 
                 'Z2_M1_T1_1', 'Z2_M1_T1_3', 'Z2_M1_T1_5']

# initial arrays
X_train_L = np.zeros((0, 16))
X_train_R = np.zeros((0, 16))
y_train = np.zeros((0, 3))

for set in training_sets:
    # load from files
    labels = np.genfromtxt(os.path.join(
        data_root, set, 'labels_30hz.txt'), delimiter=',')
    coordinates_L = pd.read_hdf(os.path.join(
        model_root, f'{set}_L_h264{model_name}.h5'))
    coordinates_R = pd.read_hdf(os.path.join(
        model_root, f'{set}_R_h264{model_name}.h5'))

    # unify size
    frames = min(len(labels), len(coordinates_L), len(coordinates_R))

    # drop and convert
    X_train_L = np.vstack((X_train_L,
                          coordinates_L.filter(
                              regex='^(?!.*likelihood).*$', axis=1).to_numpy()[:frames]))
    X_train_R = np.vstack((X_train_R,
                          coordinates_L.filter(
                              regex='^(?!.*likelihood).*$', axis=1).to_numpy()[:frames]))
    y_train = np.vstack((y_train, labels[:frames, 6:9]))


In [ ]:
X_train_L.shape, X_train_R.shape, y_train.shape

# Trainning

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
net = FCNN(input_dim=X_train_L.shape[1], hidden_dim=64, output_dim=y_train.shape[1], device=device)
net.train(X_train_R, X_train_L, y_train, epochs=1000)


# Testing

In [ ]:
# pre setting training sets
test_sets = ['C_M1_T1_2', 'C_M1_T1_4',
             'R2_M1_T1_2', 'R2_M1_T1_4',
             'L2_M1_T1_2', 'L2_M1_T1_4',
             'Z2_M1_T1_2',]

# initial arrays
X_test_L = np.zeros((0, 16))
X_test_R = np.zeros((0, 16))
y_test = np.zeros((0, 3))

for set in test_sets:
    # load from files
    labels = np.genfromtxt(os.path.join(
        data_root, set, 'labels_30hz.txt'), delimiter=',')
    coordinates_L = pd.read_hdf(os.path.join(
        model_root, f'{set}_L_h264{model_name}.h5'))
    coordinates_R = pd.read_hdf(os.path.join(
        model_root, f'{set}_R_h264{model_name}.h5'))

    # unify size
    frames = min(len(labels), len(coordinates_L), len(coordinates_R))

    # drop and convert
    X_test_L = np.vstack((X_test_L,
                          coordinates_L.filter(
                              regex='^(?!.*likelihood).*$', axis=1).to_numpy()[:frames]))
    X_test_R = np.vstack((X_test_R,
                          coordinates_L.filter(
                              regex='^(?!.*likelihood).*$', axis=1).to_numpy()[:frames]))
    y_test = np.vstack((y_test, labels[:frames, 6:9]))


In [ ]:
X_test_L.shape, X_test_R.shape, y_test.shape

In [ ]:
from sklearn.metrics import mean_squared_error

y_pred = net.predict(X_test_R, X_test_L)

mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error on test data: {mse}')